In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-87d69819-955c-a370-ac12-78768cde6bb4)


### Code for all important tasks

In [2]:
import tensorflow as tf

# Create a function to import an image and resize it to be able to be used with our model
def load_and_prep_image(filename, img_shape=224, scale=True):
  """
  Reads in an image from filename, turns it into a tensor and reshapes into
  (224, 224, 3).

  Parameters
  ----------
  filename (str): string filename of target image
  img_shape (int): size to resize target image to, default 224
  scale (bool): whether to scale pixel values to range(0, 1), default True
  """
  # Read in the image
  img = tf.io.read_file(filename)
  # Decode it into a tensor
  img = tf.image.decode_jpeg(img)
  # Resize the image
  img = tf.image.resize(img, [img_shape, img_shape])
  if scale:
    # Rescale the image (get all values between 0 and 1)
    return img/255.
  else:
    return img

# Note: The following confusion matrix code is a remix of Scikit-Learn's
# plot_confusion_matrix function - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.plot_confusion_matrix.html
import itertools
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix

# Our function needs a different name to sklearn's plot_confusion_matrix
def make_confusion_matrix(y_true, y_pred, classes=None, figsize=(10, 10), text_size=15, norm=False, savefig=False):
  """Makes a labelled confusion matrix comparing predictions and ground truth labels.

  If classes is passed, confusion matrix will be labelled, if not, integer class values
  will be used.

  Args:
    y_true: Array of truth labels (must be same shape as y_pred).
    y_pred: Array of predicted labels (must be same shape as y_true).
    classes: Array of class labels (e.g. string form). If `None`, integer labels are used.
    figsize: Size of output figure (default=(10, 10)).
    text_size: Size of output figure text (default=15).
    norm: normalize values or not (default=False).
    savefig: save confusion matrix to file (default=False).

  Returns:
    A labelled confusion matrix plot comparing y_true and y_pred.

  Example usage:
    make_confusion_matrix(y_true=test_labels, # ground truth test labels
                          y_pred=y_preds, # predicted labels
                          classes=class_names, # array of class label names
                          figsize=(15, 15),
                          text_size=10)
  """
  # Create the confustion matrix
  cm = confusion_matrix(y_true, y_pred)
  cm_norm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis] # normalize it
  n_classes = cm.shape[0] # find the number of classes we're dealing with

  # Plot the figure and make it pretty
  fig, ax = plt.subplots(figsize=figsize)
  cax = ax.matshow(cm, cmap=plt.cm.Blues) # colors will represent how 'correct' a class is, darker == better
  fig.colorbar(cax)

  # Are there a list of classes?
  if classes:
    labels = classes
  else:
    labels = np.arange(cm.shape[0])

  # Label the axes
  ax.set(title="Confusion Matrix",
         xlabel="Predicted label",
         ylabel="True label",
         xticks=np.arange(n_classes), # create enough axis slots for each class
         yticks=np.arange(n_classes),
         xticklabels=labels, # axes will labeled with class names (if they exist) or ints
         yticklabels=labels)

  # Make x-axis labels appear on bottom
  ax.xaxis.set_label_position("bottom")
  ax.xaxis.tick_bottom()

  # Set the threshold for different colors
  threshold = (cm.max() + cm.min()) / 2.

  # Plot the text on each cell
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    if norm:
      plt.text(j, i, f"{cm[i, j]} ({cm_norm[i, j]*100:.1f}%)",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)
    else:
      plt.text(j, i, f"{cm[i, j]}",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)

  # Save the figure to the current working directory
  if savefig:
    fig.savefig("confusion_matrix.png")

# Make a function to predict on images and plot them (works with multi-class)
def pred_and_plot(model, filename, class_names):
  """
  Imports an image located at filename, makes a prediction on it with
  a trained model and plots the image with the predicted class as the title.
  """
  # Import the target image and preprocess it
  img = load_and_prep_image(filename)

  # Make a prediction
  pred = model.predict(tf.expand_dims(img, axis=0))

  # Get the predicted class
  if len(pred[0]) > 1: # check for multi-class
    pred_class = class_names[pred.argmax()] # if more than one output, take the max
  else:
    pred_class = class_names[int(tf.round(pred)[0][0])] # if only one output, round

  # Plot the image and predicted class
  plt.imshow(img)
  plt.title(f"Prediction: {pred_class}")
  plt.axis(False);

import datetime

def create_tensorboard_callback(dir_name, experiment_name):
  """
  Creates a TensorBoard callback instand to store log files.

  Stores log files with the filepath:
    "dir_name/experiment_name/current_datetime/"

  Args:
    dir_name: target directory to store TensorBoard log files
    experiment_name: name of experiment directory (e.g. efficientnet_model_1)
  """
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir
  )
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback

# Plot the validation and training data separately
import matplotlib.pyplot as plt

def plot_loss_curves(history):
  """
  Returns separate loss curves for training and validation metrics.

  Args:
    history: TensorFlow model History object (see: https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/History)
  """
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']

  epochs = range(len(history.history['loss']))

  # Plot loss
  plt.plot(epochs, loss, label='training_loss')
  plt.plot(epochs, val_loss, label='val_loss')
  plt.title('Loss')
  plt.xlabel('Epochs')
  plt.legend()

  # Plot accuracy
  plt.figure()
  plt.plot(epochs, accuracy, label='training_accuracy')
  plt.plot(epochs, val_accuracy, label='val_accuracy')
  plt.title('Accuracy')
  plt.xlabel('Epochs')
  plt.legend();

def compare_historys(original_history, new_history, initial_epochs=5):
    """
    Compares two TensorFlow model History objects.

    Args:
      original_history: History object from original model (before new_history)
      new_history: History object from continued model training (after original_history)
      initial_epochs: Number of epochs in original_history (new_history plot starts from here)
    """

    # Get original history measurements
    acc = original_history.history["accuracy"]
    loss = original_history.history["loss"]

    val_acc = original_history.history["val_accuracy"]
    val_loss = original_history.history["val_loss"]

    # Combine original history with new history
    total_acc = acc + new_history.history["accuracy"]
    total_loss = loss + new_history.history["loss"]

    total_val_acc = val_acc + new_history.history["val_accuracy"]
    total_val_loss = val_loss + new_history.history["val_loss"]

    # Make plots
    plt.figure(figsize=(8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(total_acc, label='Training Accuracy')
    plt.plot(total_val_acc, label='Validation Accuracy')
    plt.plot([initial_epochs-1, initial_epochs-1],
              plt.ylim(), label='Start Fine Tuning') # reshift plot around epochs
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(2, 1, 2)
    plt.plot(total_loss, label='Training Loss')
    plt.plot(total_val_loss, label='Validation Loss')
    plt.plot([initial_epochs-1, initial_epochs-1],
              plt.ylim(), label='Start Fine Tuning') # reshift plot around epochs
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.xlabel('epoch')
    plt.show()

# Create function to unzip a zipfile into current working directory
# (since we're going to be downloading and unzipping a few files)
import zipfile

def unzip_data(filename):
  """
  Unzips filename into the current working directory.

  Args:
    filename (str): a filepath to a target zip folder to be unzipped.
  """
  zip_ref = zipfile.ZipFile(filename, "r")
  zip_ref.extractall()
  zip_ref.close()

# Walk through an image classification directory and find out how many files (images)
# are in each subdirectory.
import os

def walk_through_dir(dir_path):
  """
  Walks through dir_path returning its contents.

  Args:
    dir_path (str): target directory

  Returns:
    A print out of:
      number of subdiretories in dir_path
      number of images (files) in each subdirectory
      name of each subdirectory
  """
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
      y_true: true labels in the form of a 1D array
      y_pred: predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

### Get the data

In [3]:
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

--2024-06-03 01:20:08--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.195.207, 173.194.196.207, 173.194.197.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.195.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2024-06-03 01:20:08 (94.3 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



### Visualizing the text data

In [4]:
import pandas as pd
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train_df.head() #1 is disaster, 0 is not

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
# Shuffle training data
train_df_shuffled = train_df.sample(frac=1, random_state=42) # frac for what percent of data (here 1 means 100%)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [6]:
# How many samples of each class?
train_df_shuffled.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [7]:
# visualize some random data
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(disaster)" if target > 0 else "(not disaster)")
  print(f"Text:\n{text}\n")

Target: 0 (not disaster)
Text:
The Drowning Girl by Caitlin R. Kiernan Centipede Press Signed numbered Limited - Full reÛ_ http://t.co/m2YUXNqlqY http://t.co/V8GKkfMFXT

Target: 0 (not disaster)
Text:
@hashtagteaclub place out. It was ex Pars defender Andy Tod in full uniform. It was instant pandemonium. Utter mayhem ensued!

Target: 0 (not disaster)
Text:
@Weapon_X_music hey guys thanks for a rock in my world and for the follow????????

Target: 1 (disaster)
Text:
@DArchambau THX for your great encouragement and for RT of NEW VIDEO http://t.co/cybKsXHF7d The Coming Apocalyptic US Earthquake &amp; Tsunami

Target: 0 (not disaster)
Text:
Rationing of food and water may also become necessary during an emergency such as a natural disaster or terror attack.



### Split into train and validation tests

In [8]:
from sklearn.model_selection import train_test_split
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                                            train_df_shuffled['target'].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [9]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [10]:
train_sentences[:10]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk',
       '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
       'destroy the free fandom honestly',
       'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
       '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
       'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
      dtype=object)

### Need to convert texts into numbers.
Tokenization vs. Embedding
1. Tokenization - every number will be labelled(direct mapping) from 0 to end(like how one-hot encoding works) but its too time taking and complex for texts.
2. Embedding - Every words will be converted into vectors. Matrix of feature vectors. Depending on size be 128 dimensional vector, 512 vector etc, also gets updated every time neural network keeps learning. Can be created or use pre-learned embedding, like Word2Vec, GloVe etc.

### 1. Text Vectorization(Tokenization)
1. Standardize each sample(lowercase + punctuation stripping)
2. Split each sample into substrings(usually words)
3. recombine substrings into tokens(ngrams)
4. index tokens
5. transform using index into vectors

In [11]:
# Text Vectorization(Tokenization)
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [12]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

#use the default variables
text_vectorizer = TextVectorization(max_tokens=None, #Here None means no cap, that there is no particular number of occurences specified for any word
                                    standardize='lower_and_strip_punctuation',
                                    split='whitespace',
                                    ngrams=None, #treat every token of its own, instead of groups
                                    output_mode='int', #maps token into integer vectors
                                    output_sequence_length=None) #shorter words gets padded to zero, if None according to the max length


In [13]:
#average number of words in the sentences
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [14]:
# Setup custom text vectorization variables
max_vocab_length = 10000 #max number of words to have in the vocabulary
max_length = 15 #max length the sequences will be

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [15]:
#Fit the text vectorizer to the training sentences
text_vectorizer.adapt(train_sentences)

In [16]:
random_sentence = random.choice(train_sentences)
print(f"Original Text:\n{random_sentence}\
      \n\nVectorized version of the text:")
text_vectorizer([random_sentence])

Original Text:
Crying out for more! Set me ablaze      

Vectorized version of the text:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[1693,   36,   10,   51,  284,   31,  715,    0,    0,    0,    0,
           0,    0,    0,    0]])>

In [17]:
# get the unique words in vocabulary after its fit for tokenizer
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]

print(f"Number of words in vocabulary: {len(words_in_vocab)}")
print(f"Most common words: {top_5_words}")
print(f"Least common words: {bottom_5_words}")

Number of words in vocabulary: 10000
Most common words: ['', '[UNK]', 'the', 'a', 'in']
Least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### 2. Embedding Layer

Tensorflow's predefined embedding layer-
* `input_dim` - size of vocabulary
* `output_dim` - size of output embedding layer as matrix
* `embeddings_initializer` - length of sequences being passed
* `input_length` - length of sequences being passed to embedding layer

In [18]:
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             embeddings_initializer='uniform',
                             input_length=max_length #how long is each input
                             )
embedding

In [19]:
random_sentence = random.choice(train_sentences)

print(f"Original Text:\n{random_sentence}\
      \n\nEmbedded version of the text:\n")

sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original Text:
MTR issues Hazardous Weather Outlook (HWO) http://t.co/tGLK2UUs2Z      

Embedded version of the text:



<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.0168146 , -0.04274965, -0.02360236, ...,  0.03940842,
         -0.01809726, -0.04327699],
        [-0.03827324, -0.0157344 , -0.03140693, ...,  0.02724015,
         -0.03277856, -0.04413229],
        [-0.02030146,  0.01288385,  0.01503709, ..., -0.00821873,
         -0.02507216,  0.02504537],
        ...,
        [-0.01783335, -0.00037882,  0.00632449, ...,  0.00772839,
          0.01527565, -0.04540072],
        [-0.01783335, -0.00037882,  0.00632449, ...,  0.00772839,
          0.01527565, -0.04540072],
        [-0.01783335, -0.00037882,  0.00632449, ...,  0.00772839,
          0.01527565, -0.04540072]]], dtype=float32)>

In [20]:
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.0168146 , -0.04274965, -0.02360236, -0.0155329 , -0.02337146,
         0.04229205,  0.02497843,  0.03037209,  0.03892723, -0.04588979,
         0.00772374, -0.01398348,  0.03257105, -0.0317153 , -0.03528474,
         0.04264912,  0.00155728, -0.0064554 , -0.04252857, -0.00218674,
         0.03318527,  0.01768355, -0.0018527 , -0.00863342, -0.00064357,
         0.0066146 ,  0.04134873, -0.00583138,  0.02071238, -0.01580955,
        -0.00844784,  0.03077657, -0.02775598, -0.00918003,  0.01890132,
         0.04832547,  0.04516877,  0.00062354, -0.01727848, -0.04047556,
         0.02178817,  0.03140176,  0.01287467, -0.04950379, -0.00732214,
         0.01023386,  0.04418736,  0.03431734, -0.02486646, -0.01220362,
         0.03935898, -0.03222905,  0.03055353,  0.03295746, -0.01900048,
        -0.02214911, -0.04107598,  0.02128715, -0.01529677, -0.02085267,
        -0.02745088, -0.02095828,  0.00144689, -0.02841843, -0.0313307 ,
  

### Model 0
Baseline model with Naive Bayes with TF-IDF to convert words to numbers

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

#Make pipeline
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()), #converts words to numbers
    ("clf", MultinomialNB()) #model the text
])

#Fit the pipeline on the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [22]:
#Evaluate the model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Baseline Model Accuracy: {baseline_score*100:.2f}%")

Baseline Model Accuracy: 79.27%


In [23]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

Evaluation metrics

In [24]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  model_accuracy = accuracy_score(y_true, y_pred)*100
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [25]:
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 1
FNN model - A simple dense model

In [26]:
#Directory to save tensorboard logs
SAVE_DIR = "model_logs"

In [27]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string") #inputs are 1 dimensional strings

x = text_vectorizer(inputs) #Turn the input texts into numbers
x = embedding(x) #create embedding of the numbers

#x = layers.Flatten()(x) #to reshape to the size of labels
x = layers.GlobalAveragePooling1D()(x) #reduce sequence dimension to match the shape of labels

outputs = layers.Dense(1, activation='sigmoid')(x) #sigmoid activation for binary

model_1 = tf.keras.Model(inputs, outputs, name='model_1_dense')


In [28]:
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [29]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [30]:
model_1_history = model_1.fit(train_sentences,
                              train_labels,
                              epochs=10,
                              validation_data =(val_sentences,val_labels),
                              callbacks = [create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                      experiment_name = 'model_1_dense')])

Saving TensorBoard log files to: model_logs/model_1_dense/20240603-012017
Epoch 1/10
215/215 [==============================] - 17s 60ms/step - loss: 0.6118 - accuracy: 0.6938 - val_loss: 0.5360 - val_accuracy: 0.7638
Epoch 2/10
215/215 [==============================] - 2s 8ms/step - loss: 0.4413 - accuracy: 0.8208 - val_loss: 0.4684 - val_accuracy: 0.7861
Epoch 3/10
215/215 [==============================] - 2s 7ms/step - loss: 0.3459 - accuracy: 0.8613 - val_loss: 0.4542 - val_accuracy: 0.7966
Epoch 4/10
215/215 [==============================] - 2s 9ms/step - loss: 0.2832 - accuracy: 0.8933 - val_loss: 0.4667 - val_accuracy: 0.7874
Epoch 5/10
215/215 [==============================] - 2s 10ms/step - loss: 0.2360 - accuracy: 0.9108 - val_loss: 0.4843 - val_accuracy: 0.7808
Epoch 6/10
215/215 [==============================] - 2s 8ms/step - loss: 0.2004 - accuracy: 0.9288 - val_loss: 0.5059 - val_accuracy: 0.7795
Epoch 7/10
215/215 [==============================] - 2s 8ms/step - los

In [31]:
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.6095 - accuracy: 0.7769


[0.6094948053359985, 0.7769029140472412]

In [32]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 [==============================] - 0s 2ms/step


(762, 1)

In [33]:
model_1_pred_probs[:10]

array([[0.46823198],
       [0.8638187 ],
       [0.99952567],
       [0.07398148],
       [0.00951716],
       [0.9805171 ],
       [0.8865492 ],
       [0.99971205],
       [0.99878865],
       [0.5128178 ]], dtype=float32)

In [34]:
#Convert model prediction probabilities to label format so <0.5 is 0(not disaster) and >0.5 is 1(disaster)
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [35]:
model_1_results = calculate_results(y_true=val_labels,
                                     y_pred=model_1_preds)
model_1_results

{'accuracy': 77.69028871391076,
 'precision': 0.7775711689884918,
 'recall': 0.7769028871391076,
 'f1': 0.7754662769426549}

In [36]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [37]:
# function to compare baseline results to new model results
def compare_baseline_to_new_results(baseline_results, new_model_results):
  for key, value in baseline_results.items():
    print(f"Baseline {key}: {value:.2f}, New {key}: {new_model_results[key]:.2f}, Difference: {new_model_results[key]-value:.2f}")

compare_baseline_to_new_results(baseline_results=baseline_results,
                                new_model_results=model_1_results)

Baseline accuracy: 79.27, New accuracy: 77.69, Difference: -1.57
Baseline precision: 0.81, New precision: 0.78, Difference: -0.03
Baseline recall: 0.79, New recall: 0.78, Difference: -0.02
Baseline f1: 0.79, New f1: 0.78, Difference: -0.01


#Visualize learned embeddings

In [39]:
#Get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:5]

(10000, ['', '[UNK]', 'the', 'a', 'in'])

In [40]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [43]:
#Get the weight matrix of the embedding layer
embed_weights = model_1.get_layer('embedding').get_weights()[0]
print(embed_weights.shape)

(10000, 128)


In [44]:
# # # Code below is adapted from: https://www.tensorflow.org/tutorials/text/word_embeddings#retrieve_the_trained_word_embeddings_and_save_them_to_disk
# import io

# # Create output writers
# out_v = io.open("embedding_vectors.tsv", "w", encoding="utf-8")
# out_m = io.open("embedding_metadata.tsv", "w", encoding="utf-8")

# # Write embedding vectors and words to file
# for num, word in enumerate(words_in_vocab):
#   if num == 0:
#      continue # skip padding token
#   vec = embed_weights[num]
#   out_m.write(word + "\n") # write words to file
#   out_v.write("\t".join([str(x) for x in vec]) + "\n") # write corresponding word vector to file
# out_v.close()
# out_m.close()

# # Download files locally to upload to Embedding Projector
# try:
#   from google.colab import files
# except ImportError:
#   pass
# else:
#   files.download("embedding_vectors.tsv")
#   files.download("embedding_metadata.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>